In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

csv_output_dir = "CSVs"

df = pd.read_csv(csv_output_dir + "/distortion_results.csv")  # Replace with actual path


df['Value'] = df['Value'].round(5)

# inverting lpips
lpips_mask = df['Metric'] == 'LPIPS'
if lpips_mask.any():
    lpips_values = df.loc[lpips_mask, 'Value']
    lpips_min = lpips_values.min()
    lpips_max = lpips_values.max()
    if lpips_max > lpips_min:
        df.loc[lpips_mask, 'Value'] = 1 - (lpips_values - lpips_min) / (lpips_max - lpips_min)
    else:
        # All values are the same – invert the constant
        df.loc[lpips_mask, 'Value'] = 1 - lpips_values

# setting 'inf' to 1.0
psnr_mask = df['Metric'] == 'PSNR'
if psnr_mask.any():
    psnr_values = df.loc[psnr_mask, 'Value']

    # Identify finite vs inf values
    is_finite = psnr_values.apply(lambda x: pd.notna(x) and x != float('inf') and x != float('Inf'))
    finite_psnr = psnr_values[is_finite]

    psnr_max = finite_psnr.max()

    # Set 'inf' values to 1.0 directly
    df.loc[psnr_mask & ~is_finite, 'Value'] = psnr_max


distortion_types = ['Noise', 'Brightness', 'Shift', 'Blur']
metrics = df['Metric'].unique()

# Normalize values relative to DistortionValue == 0 per Picture, Metric, Method, DistortionType
def normalize_group(group):
    min_val = group['DistortionValue'].min()
    base = group[group['DistortionValue'] == min_val]['Value'] 
    if not base.empty and base.values[0] != 0:
        group['NormalizedValue'] = group['Value'] / base.values[0]
        
    else:
        group['NormalizedValue'] = float('nan')  # avoid division by zero
    return group

df = df.groupby(['Picture', 'DistortionType', 'Metric'], group_keys=False).apply(normalize_group)

# averaging values across picture types
avg_df = df.groupby(['DistortionType', 'Metric', 'DistortionValue'], as_index=False)['NormalizedValue'].mean()


fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, distortion in enumerate(distortion_types):
    ax = axes[idx]
    subset = avg_df[avg_df['DistortionType'] == distortion]

    for metric in metrics:
        metric_data = subset[subset['Metric'] == metric]
        metric_data_sorted = metric_data.sort_values('DistortionValue')

        ax.plot(metric_data_sorted['DistortionValue'],
                metric_data_sorted['NormalizedValue'],
                label=metric,
                marker='o')

    ax.set_title(f'{distortion}')
    ax.set_xlabel('Distortion Parameter')
    ax.set_ylabel('Avgerave Normalized Metric Value')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()